# Example for displaying nodal displacements from GeniE B1 Tutorial

## Introduction
This example demonstrates how SifIO can be used through Python to print the nodal dispacements for a given node in a table. A Sesam Manager example model is provided where we run Sestra to create a SIN and a SIF file.
Firstly, you need to run Sestra analysis as usual and get the result file, then you can use SifIO Python package to get the data in the result file for further post-processing in Python.  The procedure is explained below.

### Model
Model from GeniE B1 tutorial will be used. Below is the nodal displacements of the node 13 in LC 1 when viewed with Xtract:
<div>
<img src="_imgs/model.png" width="900"/>
</div>


## Prerequisities 

### Install SifIO
If you haven't already, install the SifIO Python package by running the code below

In [ ]:
%pip install -r sifio-requirements.txt

## Workflow
Firstly, Sestra is run to create the needed Sesam result files.

Note: The relevant SIN and SIF files are already provided, so it's not necessary to run the Sesam Manager example below to run the SifIO example further down.



### Optional: Run analysis with Sesam Manager
A Sesam Manager example is provided. The workflow sketched below will run Sestra twice to create a SIN file and SIF file (During debugging it is better to use SIF format as it is in ASCII format and therefore human-readable)
<div>
<img src="_imgs/sesamManager.png" width="300"/>
</div>
Sesam Manager is run from the code block below. (Use the play button to run it)


In [ ]:
import subprocess
import os
import shutil

# Make sure the path to Sesam Manager is correct
sesam_manager_path = r"C:\Program Files (x86)\DNV\Sesam Manager V6.7-00\Bin\DNVS.Sesam.Manager.Win.exe"

# This path must match Workspace folder setting in Sesam Manager
dnv_working_directory = "C:\\DNV\\Workspaces\\"

# Set to True if you want to run Sesam Manager in batch mode, or False if you want to run interactively
batch = True

manager_example_name = "SM_SestraRun"

cwd = os.getcwd()
try:
    os.chdir("SesamManagerModel")
    command = [sesam_manager_path,  manager_example_name, "/com=SM_SestraRun.js",  "/new"]
    if batch:
        command.append("/batch")
    result = subprocess.run(command)
    if result.returncode != 0:
        raise Exception("Sesam Manager failed with return code: " + str(result.returncode))
    repository_folder = os.path.join(dnv_working_directory, manager_example_name,"_repository")
except Exception as e:
    print("An error has occurred, please check any log files for more information")
    raise
finally:
    os.chdir(cwd)

print("Sesam Workspace repository path: " + repository_folder)
print("Files produced from Sesam Manager workflow:" + str(os.listdir(repository_folder)))

## Copy the SIN file created by Sestra above, to the current directory
copied=shutil.copy(repository_folder + "\\R1.SIN", ".")
print("Copied:  " + copied)
## Copy the SIF file created by Sestra above, to the current directory
copied=shutil.copy(repository_folder + "\\R1.SIF", ".")
print("Copied:  " + copied)

R1.SIN and R1.SIF files should appear in the current directory.

### SifIO
Run the following code block to get the displacement data from the R1.SIN file. You can also read the R1.SIF file.

In [ ]:
import dnv.net.runtime
from dnv.sesam.sifapi.core import ISifData
from dnv.sesam.sifapi.io import SesamDataFactory

"""Reads nodal displacement data (RVNODDIS) of a result case (LC 1) and FE node (node 13) in the R1.SIN file"""
with SesamDataFactory.CreateReader(".", "R1.SIN") as adapter:
    adapter.CreateModel()

    ires = 1  # result case reference number
    iinod = 13  # internal node number

    data = adapter.ReadExt('RVNODDIS', [ires, iinod])
    print(f'Length of the data: {len(data)}')
    print(f'Data type: {type(data)}')

    # Convert System.Double[] to a Python list
    data_list = [element for element in data]
    print(data_list)  # print all the data in the RVNODDIS

    # Put displacement x, y, z, rx, ry, rz into separate variables and print
    disp_x = data_list[6]
    disp_y = data_list[7]
    disp_z = data_list[8]
    disp_rx = data_list[9]
    disp_ry = data_list[10]
    disp_rz = data_list[11]

    print(f'\nNodal displacement for result case {ires}, node {iinod}:')
    print(f'X  = {disp_x:.2E} m')
    print(f'Y  = {disp_y:.2E} m')
    print(f'Z  = {disp_z:.2E} m')
    print(f'RX = {disp_rx:.2E} rad')
    print(f'RY = {disp_ry:.2E} rad')
    print(f'RZ = {disp_rz:.2E} rad')

You should get the same displacement values shown in Xtract.

#### Regarding data ID and index
To get the correct RVNODDIS data, you must refer to Sesam results interface format documentation and see which IDs are needed to be referred (https://sesam.dnv.com/download/windows64/sesam_results_interface_format.pdf):
<div>
<img src="_imgs/rvnoddis.png" width="700"/>
</div>

For the entries contained in RVNODDIS data, refer to the index 1 to 11 below. The number of the data entries varies between data types so you have to look at the Sesam results interface format documentation and the SIF file for debugging.
<div>
<img src="_imgs/r1sif.png" width="700"/>
</div>

#### Documentation
Please see the following documentation for a complete overview of the SIF data specification:
* Sesam Input Interface Format: https://sesam.dnv.com/download/windows64/sesam_input_interface_format.pdf
* Sesam Results Interface Format: https://sesam.dnv.com/download/windows64/sesam_results_interface_format.pdf


#### Looping through IDs
To loop through all the result cases and all the nodes, you can either (1) loop manually through "IRES" and "IINOD" in Python, or (2) use ReadAll() method.